In [329]:
%matplotlib inline

# Familiar libraries.
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
import time
 

# Take a moment to install Theano.  We will use it for building neural networks.
import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print theano.config.device # We're using CPUs (for now)
print theano.config.floatX # Should be 64 bit for CPUs

np.random.seed(0)

cpu
float64


In [164]:
train_data = np.genfromtxt("training.csv", delimiter=",", skip_header=1, usecols=(30)
                           , converters={30: lambda x : x.replace(" ", ",")})
train_data = np.array(map(lambda x : np.array(x.split(",")).astype(int), train_data))              
train_labels = np.genfromtxt("training.csv", delimiter=",", skip_header=1, usecols=np.arange(30))
shuffle = np.random.permutation(np.arange(len(train_data)))
train_data, train_labels = train_data[shuffle], train_labels[shuffle]
test_data, test_labels = train_data[6000:], train_labels[6000:]
train_data, train_labels = train_data[:1000], train_labels[:1000]
print "done"

done


In [4]:
train_l_eye_center_x, test_l_eye_center_x = train_labels[:,1], test_labels[:,1]
c1 = ~np.isnan(train_l_eye_center_x)
c2 = ~np.isnan(test_l_eye_center_x)
train_l_eye_center_x, test_l_eye_center_x = train_l_eye_center_x[c1], test_l_eye_center_x[c2]
train_data_l_eye_x, test_data_l_eye_x = train_data[c1], test_data[c2]
print "done"


done


In [23]:
lr = LinearRegression()
print "start fitting"
lr.fit(train_data_l_eye_x, train_l_eye_center_x)
print "done fitting"
preds = lr.predict(test_data_l_eye_x)
print lr.score(test_data_l_eye_x, test_l_eye_center_x)
print test_l_eye_center_x[:10]
print preds[:10]
print np.sqrt(mean_squared_error(test_l_eye_center_x, preds))
print "done"

start fitting
done fitting
0.0253662183391
[ 35.85741972  38.40062136  40.06935652  37.03348966  38.81603478
  37.60270439  34.64367724  38.29228615  37.66240569  38.68093585]
[ 37.33464166  37.84373053  41.31575921  36.65311362  39.94004226
  31.58072542  34.01099241  36.76979507  40.03090236  35.82455776]
3.19683970631
done


In [12]:
cross_validation.cross_val_score(LinearRegression(), train_data_l_eye_x, y=train_l_eye_center_x, scoring='root_mean_squared_error', cv=2)

array([ -1.22510959e+21,  -1.32272154e+20])

In [30]:
train_data, train_labels = train_data[:2000], train_labels[:2000]
feature_lrs = []
i = 0
while i < 30:
    feature_train_labels = train_labels[:,i]
    remove_nan = ~np.isnan(feature_train_labels)
    feature_train_labels = feature_train_labels[remove_nan]
    feature_train_data = train_data[remove_nan]
    feature_lr = LinearRegression()
    feature_lr.fit(feature_train_data, feature_train_labels)
    feature_lrs.append(feature_lr)
    i += 1
    
print feature_lrs
    
    

[LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=True, normalize=False), LinearRegression(copy_X=True, fit_intercept=Tru

In [31]:
feature_rmses = []
i = 0
while i < 30:
    feature_test_labels = test_labels[:,i]
    remove_nan = ~np.isnan(feature_test_labels)
    feature_test_labels = feature_test_labels[remove_nan]
    feature_test_data = test_data[remove_nan]
    feature_lr = feature_lrs[i]
    feature_preds = feature_lr.predict(feature_test_data)
    feature_rmse = np.sqrt(mean_squared_error(feature_test_labels, feature_preds))
    feature_rmses.append(feature_rmse)
    i += 1
    
print feature_rmses
print np.mean(feature_rmses)
    

[4.4203886232471703, 3.1968397063149112, 4.2862249439474276, 3.1955835531876322, 2.0726150670760624, 1.7969938165709884, 2.9099751087269778, 2.28886112014274, 2.5992171935264734, 1.827160626688296, 3.1753097212102461, 2.2332240919455635, 3.0601995390057537, 2.7236461357891226, 2.6147127653263307, 3.2500418290126922, 3.110722752397713, 2.730710604170226, 3.2320196556391667, 3.2003935435194761, 5.0799806807228167, 6.4531159171281258, 3.3852377119142876, 3.8839393517979452, 3.7104300601205602, 3.8419574508393786, 3.1286389401652128, 3.8080587545984765, 5.0991290140629628, 6.1987427687862136]
3.41713570159


In [34]:
print len(train_data)

1000


In [236]:
def tNeuralNetwork(training_data, training_labels, testing_data):
    # Try playing with this value.
    numHiddenNodes = 600 
    numFeatures = training_data[1].size
    w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
    w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, 1))*.01)))
    params = [w_1, w_2]


    ## (2) Model
    X = T.matrix("X")
    Y = T.vector("Y")
    # Two notes:
    # First, feed forward is the composition of layers (dot product + activation function)
    # Second, activation on the hidden layer still uses sigmoid
    def model(X, w_1, w_2):
        return T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2)
    y_hat = model(X, w_1, w_2)
    

    ## (3) Cost...same as linear regression
    cost = T.mean((y_hat - Y)**2)

    ## (4) Minimization.  Update rule changes to backpropagation.
    alpha = 0.0001
    
    def backprop(cost, w):
        grads = T.grad(cost=cost, wrt=w)
        updates = []
        for w1, grad in zip(w, grads):
            updates.append([w1, w1 - grad * alpha])
        return updates
    
    update = backprop(cost, params)
    train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
    predict = theano.function(inputs=[X], outputs=y_hat, allow_input_downcast=True)

    miniBatchSize = 1 
    def gradientDescentStochastic(epochs, training_data, training_labels):
        for i in range(epochs):
            for start, end in zip(range(0, len(training_data), miniBatchSize), range(miniBatchSize, len(training_data), miniBatchSize)):
                cost = train(training_data[start:end], training_labels[start:end])


    gradientDescentStochastic(5, training_data, training_labels)
    return predict(testing_data)

In [237]:
mini_train_data, mini_train_labels = train_data[:100]/255.0, train_labels[:100]/255.0
feature_pred_list = []
feature_rmses = []
i = 0
while i < 2:
    feature_train_labels = mini_train_labels[:,i]
    remove_nan = ~np.isnan(feature_train_labels)
    feature_train_labels = feature_train_labels[remove_nan]
    feature_train_data = mini_train_data[remove_nan]
    feature_test_labels = test_labels[:,i]
    remove_nan = ~np.isnan(feature_test_labels)
    feature_test_labels = feature_test_labels[remove_nan]
    feature_test_data = test_data[remove_nan]
    feature_preds = tNeuralNetwork(feature_train_data, feature_train_labels, feature_test_data/255.0)
    feature_rmse = np.sqrt(mean_squared_error(feature_test_labels, feature_preds*255.0))
    feature_rmses.append(feature_rmse)
    feature_pred_list.append(feature_preds)
    i += 1
    
print feature_rmses

[4.9753809657051464, 3.9251387419157475]


In [196]:
print train_data / 255.0

[[ 0.41568627  0.3254902   0.29019608 ...,  0.18039216  0.19607843
   0.21960784]
 [ 0.09411765  0.08235294  0.06666667 ...,  0.34117647  0.3372549
   0.34117647]
 [ 0.19215686  0.19607843  0.2        ...,  0.40784314  0.40392157
   0.41176471]
 ..., 
 [ 0.37254902  0.09019608  0.01176471 ...,  0.21568627  0.50196078
   0.48235294]
 [ 0.32156863  0.38431373  0.38039216 ...,  0.6745098   0.7254902
   0.78431373]
 [ 0.21176471  0.21568627  0.23921569 ...,  0.40392157  0.40392157
   0.40392157]]


In [174]:
def tCntxtNeuralNetwork(training_data, training_labels, testing_data):
    # Try playing with this value.
    numHiddenNodes = 600 
    numFeatures = training_data[1].size
    numLabels = training_labels[1].size
    pred_dict = {}
    weight_dict = {}
    pred_fun_dict = {}
    train_fun_dict = {}
    params = [w_1, w_2]
    for i in np.arange(numLabels):
        curr_w1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
        curr_w2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, 1))*.01)))
        weight_dict[i] = [curr_w1, curr_w2]        
        curr_pred_fun = theano.function(inputs=[X, Y], outputs=y_hat
                                        , updates=[(weight_dict[i][0], allow_input_downcast=True)

    ## (2) Model
    X = T.matrix("X")
    Y = T.vector("Y")
#     w_1 = T.matrix("w_1")
#     w_2 = T.vector("w_2")
    # Two notes:
    # First, feed forward is the composition of layers (dot product + activation function)
    # Second, activation on the hidden layer still uses sigmoid
    def model(X, w_1, w_2):
        return T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2)
    y_hat = model(X, w_1, w_2)
    

    ## (3) Cost...same as linear regression
    cost = T.mean((y_hat - Y)**2)

    ## (4) Minimization.  Update rule changes to backpropagation.
    alpha = 0.0001
    
    def backprop(cost, w):
        grads = T.grad(cost=cost, wrt=w)
        updates = []
        for w1, grad in zip(w, grads):
            updates.append([w1, w1 - grad * alpha])
        return updates
    
    update = backprop(cost, params)
    train = theano.function(inputs=[X, Y], outputs=y_hat, updates=update, allow_input_downcast=True)
    predict = theano.function(inputs=[X], outputs=y_hat, allow_input_downcast=True)

    miniBatchSize = 1 
    def gradientDescentStochastic(epochs, training_data, training_labels):
        for i in range(epochs):
            for start, end in zip(range(0, len(training_data), miniBatchSize), range(miniBatchSize, len(training_data), miniBatchSize)):
                for i in np.arange(numLabels):
                    pred = train(training_data[start:end], training_labels[start:end])


    gradientDescentStochastic(5, training_data, training_labels)
    return predict(testing_data)

In [284]:
class NNet:
    X = T.matrix("X")
    Y = T.vector("Y")
    def model(self, X, w_1, w_2):
        return T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2)
    
    def backprop(self, cost, w):
        grads = T.grad(cost=cost, wrt=w)
        updates = []
        for w1, grad in zip(w, grads):
            updates.append([w1, w1 - grad * self.alpha])
        return updates
    
    def __init__(self, feature_cnt, hidden_node_cnt):
        self.numFeatures = feature_cnt
        self.numHiddenNodes = hidden_node_cnt 
        self.w_1 = theano.shared(np.asarray((np.random.randn(*(self.numFeatures, self.numHiddenNodes))*.01)))
        self.w_2 = theano.shared(np.asarray((np.random.randn(*(self.numHiddenNodes, 1))*.01)))
        self.params = [self.w_1, self.w_2]
        self.y_hat = self.model(self.X, self.w_1, self.w_2)
        self.cost = T.mean((self.y_hat - self.Y)**2)
        self.alpha = 0.0001
        self.update = self.backprop(self.cost, self.params)
        self.train = theano.function(inputs=[self.X, self.Y]
                                     , outputs=self.y_hat, updates=self.update, allow_input_downcast=True)
        self.predict = theano.function(inputs=[self.X], outputs=self.y_hat, allow_input_downcast=True)
        self.miniBatchSize = 1 
        
    def gradientDescentStochastic(self, epochs, training_data, training_labels):
        for i in range(epochs):
            for start, end in zip(range(0, len(training_data), self.miniBatchSize), range(self.miniBatchSize, len(training_data), self.miniBatchSize)):
                pred = self.train(training_data[start:end], training_labels[start:end])

    def fit(self, training_data, training_labels):
        self.gradientDescentStochastic(5, training_data, training_labels)
        
    def pred(self, testing_data):
        return self.predict(testing_data)
    
    def fit_step(self, training_data, training_labels, start, end):
        pred = self.train(training_data[start:end], training_labels[start:end])
        return pred
        

        
def mutualTrain(training_data, training_labels, epochs):
    nnets = []
    net_cnt = training_labels.shape[1]
    preds = np.zeros(net_cnt)
    aug_cnt = net_cnt
    img_size = training_data.shape[1]
    aug_training_data = np.zeros((training_data.shape[0], img_size + aug_cnt))
    aug_training_data[:,:-aug_cnt] = training_data
    for i in range(net_cnt):
        nnet = NNet(img_size + aug_cnt, 600)
        nnets.append(nnet)
    for i in range(epochs):
        for start, end in zip(range(0, len(aug_training_data), 1), range(1, len(aug_training_data), 1)):
            for j in range(net_cnt):
                curr_net = nnets[j]
                pred = curr_net.fit_step(aug_training_data, training_labels[j], start, end)
                aug_training_data[start, j] = pred
    return nnets


In [271]:
mini_train_data, mini_train_labels = train_data[:100]/255.0, train_labels[:100]/255.0
feature_pred_list = []
feature_rmses = []
i = 0
while i < 1:
    feature_train_labels = mini_train_labels[:,i]
    remove_nan = ~np.isnan(feature_train_labels)
    feature_train_labels = feature_train_labels[remove_nan]
    feature_train_data = mini_train_data[remove_nan]
    feature_test_labels = test_labels[:,i]
    remove_nan = ~np.isnan(feature_test_labels)
    feature_test_labels = feature_test_labels[remove_nan]
    feature_test_data = test_data[remove_nan]
    feature_nn = NNet(feature_train_data[0].size, 600)
    print feature_nn.w_1.eval()
    feature_nn.fit(feature_train_data, feature_train_labels)
#     print feature_nn.w_1.eval()
    feature_preds = feature_nn.pred(feature_test_data/255.0)
    feature_rmse = np.sqrt(mean_squared_error(feature_test_labels, feature_preds*255.0))
    feature_rmses.append(feature_rmse)
    feature_pred_list.append(feature_preds)
    i += 1
    
print feature_rmses

[[ 0.00179916  0.01344482  0.00464214 ...,  0.00712863  0.00613196
   0.01374534]
 [-0.00015212 -0.00772643  0.00182587 ..., -0.00962728  0.00681799
   0.00433796]
 [ 0.00294586  0.00269758  0.01145068 ..., -0.00484326 -0.0016041
   0.01242837]
 ..., 
 [-0.00131788  0.00344589 -0.00263903 ..., -0.00421758 -0.00702426
   0.00293342]
 [-0.00814809  0.01645486 -0.01164123 ..., -0.00438224 -0.00650956
  -0.0031884 ]
 [-0.01905902  0.00737533 -0.00758742 ...,  0.00774335 -0.01165548
   0.00312265]]


KeyboardInterrupt: 

In [283]:
mini_train_data, mini_train_labels = train_data[:100]/255.0, train_labels[:100,:2]/255.0
nns = mutualTrain(mini_train_data, mini_train_labels, 1)
print "done"

ValueError: Input dimension mis-match. (input[1].shape[1] = 1, input[3].shape[1] = 0)
Apply node that caused the error: Elemwise{Composite{[mul(i0, i1, sub(i2, i3))]}}[(0, 1)](TensorConstant{(1, 1) of 2.0}, Alloc.0, Dot22.0, InplaceDimShuffle{x,0}.0)
Inputs shapes: [(1, 1), (1, 1), (1, 1), (1, 0)]
Inputs strides: [(8, 8), (8, 8), (8, 8), (0, 8)]
Inputs types: [TensorType(float64, (True, True)), TensorType(float64, matrix), TensorType(float64, matrix), TensorType(float64, row)]
Use the Theano flag 'exception_verbosity=high' for a debugprint of this apply node.

In [313]:
mini_train_data, mini_train_labels = train_data[:100], train_labels[:100]
feat_num = 30
train_row_cnt = mini_train_data.shape[0]
test_row_cnt = test_data.shape[0]
feature_nnets = []
feature_nnets_cntxt = []
feature_preds_train = np.zeros((train_row_cnt, feat_num))
feature_preds_test = np.zeros((test_row_cnt, feat_num))
feature_preds_test_cntxt = np.zeros((test_row_cnt, feat_num))
feature_rmses_indpndt = []

i = 0
while i < feat_num:
    feature_train_labels = mini_train_labels[:,i]
    remove_nan = ~np.isnan(feature_train_labels)
    feature_train_labels = feature_train_labels[remove_nan]
    feature_train_data = mini_train_data[remove_nan]
    feature_test_labels = test_labels[:,i]
    remove_nan = ~np.isnan(feature_test_labels)
    feature_test_labels = feature_test_labels[remove_nan]
    feature_test_data = test_data[remove_nan]
    feature_nn = NNet(feature_train_data[0].size, 600)
    feature_nn.fit(feature_train_data, feature_train_labels)
    pred_train = feature_nn.pred(mini_train_data)
    pred_test = feature_nn.pred(test_data)
    feature_pred_test = pred_test[remove_nan]
    feature_rmse_indpndt = np.sqrt(mean_squared_error(feature_test_labels, feature_pred_test))
    feature_nnets.append(feature_nn)
    feature_preds_train[:,i] = pred_train[:,0]
    feature_preds_test[:,i] = pred_test[:,0]
    feature_rmses_indpndt.append(feature_rmse_indpndt)
    i += 1


    
    



[3.6607873191580929, 3.2579264067701872, 3.3218165342085619, 3.1732604675076681, 2.1265198480384475, 2.1718428639253151, 3.5623513967445439, 3.0441125185429962, 2.3213314677281733, 2.1752059402386195, 2.9836179379488028, 3.0511166883839196, 2.9767028866344667, 2.8858544049261079, 3.8660327259073251, 3.9450146416271883, 3.3551827528498044, 2.9594311624761693, 3.4495423159795195, 4.1534258330480824, 4.4279779065832567, 5.8103364635292643, 4.048366945376876, 4.8050879967601041, 4.0301931220832863, 4.613371578671611, 3.2919278667510241, 5.3135666221849585, 4.3466421976372862, 5.4232686201887086]
[3.612652580374188, 3.2480139709701406, 3.3285485319152626, 3.1178485773102689, 2.1112482199189757, 2.2118222097370497, 3.3403449429404311, 3.1011868448811226, 2.2552604767951316, 2.1354881636579526, 2.9986754353970264, 3.0213031783943443, 2.9397921583643831, 2.8540997063329416, 3.7869883211129522, 3.9708331241079033, 3.1800832472522047, 2.9315744405624713, 3.4648756125920457, 4.1109196977338893, 4

In [335]:
feature_rmses_cntxt = []
i = 0
while i < feat_num:
    feature_train_labels = mini_train_labels[:,i]
    remove_nan = ~np.isnan(feature_train_labels)
    feature_train_data = feature_preds_train[remove_nan]
    feature_train_labels = feature_train_labels[remove_nan]
    feature_test_labels = test_labels[:,i]
    remove_nan = ~np.isnan(feature_test_labels)
    feature_test_labels = feature_test_labels[remove_nan]
    feature_test_data = feature_preds_test[remove_nan]
    feature_nn = NNet(feat_num, 300)
    feature_nn.fit(feature_train_data, feature_train_labels)
    feature_pred_test = feature_nn.pred(feature_test_data)
#     feature_svm = svm.SVR(C=.0001)
#     feature_svm.fit(feature_train_data, feature_train_labels)
#     feature_pred_test = feature_svm.predict(feature_test_data)
    feature_rmse_cntxt = np.sqrt(mean_squared_error(feature_test_labels, feature_pred_test))
    feature_nnets_cntxt.append(feature_nn)
#     feature_preds_test_cntxt.append(feat_pred_test)
    feature_rmses_cntxt.append(feature_rmse_cntxt)
    i += 1
    
print feature_rmses_indpndt
print feature_rmses_cntxt
print np.mean(feature_rmses_indpndt)
print np.mean(feature_rmses_cntxt)

[3.6607873191580929, 3.2579264067701872, 3.3218165342085619, 3.1732604675076681, 2.1265198480384475, 2.1718428639253151, 3.5623513967445439, 3.0441125185429962, 2.3213314677281733, 2.1752059402386195, 2.9836179379488028, 3.0511166883839196, 2.9767028866344667, 2.8858544049261079, 3.8660327259073251, 3.9450146416271883, 3.3551827528498044, 2.9594311624761693, 3.4495423159795195, 4.1534258330480824, 4.4279779065832567, 5.8103364635292643, 4.048366945376876, 4.8050879967601041, 4.0301931220832863, 4.613371578671611, 3.2919278667510241, 5.3135666221849585, 4.3466421976372862, 5.4232686201887086]
[3.7594844704556745, 3.2714815050065225, 3.3231320333040184, 3.1916107019892817, 2.1071531091457816, 2.2113651863534036, 3.3612177773276333, 3.1016269813072452, 2.2595738398670728, 2.1472615938881798, 3.0122372121534862, 3.0980216300146211, 2.9755852260287452, 2.8761032747660797, 3.7985187369471545, 3.9688689529699386, 3.3143985604803348, 2.929345110285527, 3.4641458688629694, 4.1496719525001708, 4

In [314]:
# print len(feature_test_labels)
# print len(feature_pred_test)
# print len(test_data)
# print len(test_labels)
# print len(test_labels[:,i])
# print feature_preds_train.shape
# print feature_pred_train[:,0].shape
# print feature_rmses_indpndt
# print feature_rmse_indpndt
# print feature_pred_test*255
# print feature_test_labels
# print feature_rmse_cntxt
print np.mean(feature_rmses_indpndt)
print np.mean(feature_rmses_cntxt)

3.61839384775
3.58597021352
